In [1]:
import pandas as pd
df = pd.read_csv(r'train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [2]:
df= df.dropna()

In [3]:
X = df.drop('label',axis = 1) ## indep features

In [4]:
## dep features
y= df['label']

In [5]:
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [6]:
X.shape

(18285, 4)

In [7]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [8]:
### Vocabulary size
voc_size=5000


In [9]:
## one hot representation
messages = X.copy()


In [10]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [11]:
messages.reset_index(inplace = True)

In [12]:

import nltk
import re
from nltk.corpus import stopwords

In [14]:
nltk.download('stopwords')
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

Streaming output truncated to the last 5000 lines.
13285
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
1

In [18]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[1600, 3024, 786, 1754, 853, 3020, 3790, 1427, 3534, 3394],
 [679, 4230, 3338, 1873, 4640, 1722, 2088],
 [4994, 2973, 4136, 890],
 [2374, 857, 137, 3262, 3608, 2173],
 [1436, 4640, 2253, 364, 407, 2447, 4640, 171, 3145, 4520],
 [2547,
  361,
  4,
  1559,
  4125,
  4991,
  968,
  1333,
  2228,
  4478,
  1922,
  2230,
  3721,
  4991,
  2088],
 [1864, 3785, 3957, 4724, 2529, 2965, 4815, 1835, 4508, 2454, 3526],
 [2414, 748, 2643, 2535, 1384, 3708, 4991, 4577, 4508, 2454, 3526],
 [4816, 1247, 578, 2133, 4950, 4819, 4623, 2528, 4991, 1421],
 [1811, 1876, 992, 874, 1397, 2286, 3297, 3994],
 [3851, 2258, 3486, 2396, 3774, 822, 1945, 3669, 4576, 1944, 807],
 [3262, 2232, 853, 4819, 4991, 1384],
 [3494, 4603, 3503, 244, 1952, 2629, 4872, 3167, 1129],
 [1969, 3850, 347, 2535, 3941, 2832, 1906, 4508, 2454, 3526],
 [2437, 4240, 415, 4060, 3556, 4508, 2454, 3526],
 [1684, 1977, 587, 4755, 2832, 2283, 11, 3293, 3686, 3960],
 [798, 4781, 4230],
 [2865, 1081, 4967, 225, 4991, 4795, 22, 2088],
 [909, 

In [19]:
## embedding representation
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 1427 3534 3394]
 [   0    0    0 ... 4640 1722 2088]
 [   0    0    0 ... 2973 4136  890]
 ...
 [   0    0    0 ... 4508 2454 3526]
 [   0    0    0 ... 2805 4883 1578]
 [   0    0    0 ... 1826 3663 1896]]


In [20]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1600,
       3024,  786, 1754,  853, 3020, 3790, 1427, 3534, 3394], dtype=int32)

In [21]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [22]:
## Creating model
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               112800    
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [23]:
len(embedded_docs),y.shape

(18285, (18285,))

In [24]:

import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [26]:

### Finally Training
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 15s 58ms/step - loss: 0.4892 - accuracy: 0.7349 - val_loss: 0.2017 - val_accuracy: 0.9112
Epoch 2/10
192/192 [==============================] - 10s 52ms/step - loss: 0.1447 - accuracy: 0.9437 - val_loss: 0.1830 - val_accuracy: 0.9271
Epoch 3/10
192/192 [==============================] - 10s 53ms/step - loss: 0.0880 - accuracy: 0.9696 - val_loss: 0.2077 - val_accuracy: 0.9228
Epoch 4/10
192/192 [==============================] - 10s 54ms/step - loss: 0.0592 - accuracy: 0.9811 - val_loss: 0.2479 - val_accuracy: 0.9155
Epoch 5/10
192/192 [==============================] - 10s 54ms/step - loss: 0.0430 - accuracy: 0.9877 - val_loss: 0.2887 - val_accuracy: 0.9168
Epoch 6/10
192/192 [==============================] - 10s 53ms/step - loss: 0.0289 - accuracy: 0.9916 - val_loss: 0.3538 - val_accuracy: 0.9162
Epoch 7/10
192/192 [==============================] - 10s 54ms/step - loss: 0.0210 - accuracy: 0.9932 - val_loss: 0.3783 - val_accuracy:

In [27]:
y_pred1=model1.predict_classes(X_test)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [28]:
from sklearn.metrics import confusion_matrix

In [29]:
confusion_matrix(y_test,y_pred1)

array([[3109,  310],
       [ 214, 2402]])

In [30]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred1)

0.9131731565865783

In [31]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred1))

              precision    recall  f1-score   support

           0       0.94      0.91      0.92      3419
           1       0.89      0.92      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035



In [32]:
## adding dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
dropout_1 (Dropout)          (None, 20, 40)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [33]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 10s 41ms/step - loss: 0.4891 - accuracy: 0.7302 - val_loss: 0.2181 - val_accuracy: 0.9128
Epoch 2/10
192/192 [==============================] - 7s 39ms/step - loss: 0.1560 - accuracy: 0.9397 - val_loss: 0.1888 - val_accuracy: 0.9200
Epoch 3/10
192/192 [==============================] - 7s 38ms/step - loss: 0.1075 - accuracy: 0.9604 - val_loss: 0.2180 - val_accuracy: 0.9027
Epoch 4/10
192/192 [==============================] - 7s 38ms/step - loss: 0.0803 - accuracy: 0.9720 - val_loss: 0.2106 - val_accuracy: 0.9185
Epoch 5/10
192/192 [==============================] - 7s 38ms/step - loss: 0.0545 - accuracy: 0.9810 - val_loss: 0.2232 - val_accuracy: 0.9210
Epoch 6/10
192/192 [==============================] - 7s 38ms/step - loss: 0.0428 - accuracy: 0.9873 - val_loss: 0.3186 - val_accuracy: 0.9056
Epoch 7/10
192/192 [==============================] - 7s 38ms/step - loss: 0.0286 - accuracy: 0.9905 - val_loss: 0.3014 - val_accuracy: 0.919

In [34]:
y_pred=model.predict_classes(X_test)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [35]:
confusion_matrix(y_test,y_pred)

array([[3074,  345],
       [ 188, 2428]])

In [36]:
accuracy_score(y_test,y_pred)

0.9116818558409279

In [37]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.94      0.90      0.92      3419
           1       0.88      0.93      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

